Task 1 : Develop a working model of Retrieval Augmented Generation (RAG) for a QA bot for a Business, leveraging the OpenAI API and a vector database (Pinecone DB).

steps:<br>
1. Install the required libraries - openai, pinecone_client, sentence_transformers, cohere, tiktoken
2. Create a pinecone client with Pinecone API key.
3. Use sentence_transformers library to convert business database from string to numerical vectors
4. Upsert these vectors into Pinecone Vector db.
5. Retrieve the document that contains information closest to the user query(regarding the business).
6. Create OpenAI object using OpenAI API key
7. Generate the response(answer to user query) with openai and the retrieved document in the step 6.

<br>
notes: <br>
1. The 2 API keys have been removed. <br>
2. the code is not currently a working model due to Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
pip install tiktoken cohere


In [ ]:
pip install --upgrade tensorflow-probability
# to resolve dependency issues

In [ ]:
pip install --upgrade typing-extensions
# to resolve dependency issues

In [ ]:
pip install --upgrade openai

  Using cached openai-1.7.0-py3-none-any.whl (224 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.6.0
    Uninstalling openai-1.6.0:
      Successfully uninstalled openai-1.6.0


In [ ]:
pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.1 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=fa58b32112e4d390d06cab35e56ddb909479e9387d2ec7d3caa7b76ffd7a33d3
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [ ]:
pip install pinecone-client


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 9.8 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer

# Load Sentence-BERT model
embedding_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Example business data
business_data = {
    'doc1': 'The company was founded in 2005 and is known for its innovative products.',
    'doc2': 'Our business strategy focuses on sustainability and customer satisfaction.',
    'doc3': 'We recently expanded our operations to new international markets.',
}


In [ ]:
import pinecone

pinecone_api_key = "PINECONE_API_KEY"
pinecone.init(api_key=pinecone_api_key, environment = "gcp-starter")

In [ ]:
# pinecone.delete_index("quickstart")
pinecone.create_index("quickstart", dimension=8, metric="euclidean")
pinecone.describe_index("quickstart")

IndexDescription(name='quickstart', metric='euclidean', replicas=1, dimension=8.0, shards=1, pods=1, pod_type='starter', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

In [ ]:
index = pinecone.Index("quickstart")

In [ ]:
import numpy as np

# Assuming your_business_data is a dictionary with document_id as keys and document_text as values
for document_id, document_text in business_data.items():
    vector = embedding_model.encode(document_text)

    ## i have reduced the size of query vector(from 384 to 8) to match the dimension of pinecone index
    vector = vector[:8] if len(vector) >= 8 else np.pad(vector, (0, 8 - len(vector)))
    vector_dict = {"id": document_id, "values": vector.tolist()}

    index.upsert([vector_dict], namespace="ns2")


In [ ]:
index.describe_index_stats()

{'dimension': 8,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [ ]:
user_query = "What is the business strategy of your company?"
query_vector = embedding_model.encode(user_query)
query_vector = query_vector[:8] if len(query_vector) >= 8 else np.pad(query_vector, (0, 8 - len(query_vector)))
query_vector_list = query_vector.tolist()

results = index.query(
    namespace="ns2",
    vector=query_vector_list,
    top_k=1,
    include_values=True
)

In [ ]:
relevant_document_ids = [res['id'] for res in results['matches']]
print(relevant_document_ids)

['doc3']


In [ ]:
relevant_documents = [business_data[doc_id] for doc_id in relevant_document_ids]


In [ ]:
from openai import OpenAI

OPENAI_API_KEY = "OPENAI_API_KEY"

client = OpenAI(api_key=OPENAI_API_KEY)

retrieved_documents_text = " ".join(relevant_documents)

response = client.chat.completions.create(
    model="tts-1",
    messages=[
    {"role": "system", "content": retrieved_documents_text},
    {"role": "user", "content": user_query}
              ],
    max_tokens=10,
    temperature=0.7,
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0,
)
answer = response.choices[0].text.strip()
print(answer)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}